In [17]:
# https://github.com/data-bootcamp-v4/lab-sql-python-connection 
# LAB | Connecting Python to SQL


Establish a connection between Python and the Sakila database.

In [ ]:
# Switch to SQLAlchemy, because of UserWarning: 
from sqlalchemy import create_engine

# Database connection settings
host = "localhost"  # host
user = "root"  #  MySQL username
password = "Malcomx1"  #  MySQL password
database = "sakila"  #  database name


# creates a SQLAlchemy engine, which serves as the entry point for interacting with the database
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

print(f"Connected to the {database} database successfully!")

Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.
month: an integer representing the month for which rental data is to be retrieved.
year: an integer representing the year for which rental data is to be retrieved.
The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

In [19]:
import pandas as pd
from sqlalchemy.engine import Engine
from sqlalchemy.sql import text

def rentals_month(engine: Engine, month: int, year: int) -> pd.DataFrame:
    """
    Retrieves rental data for a specified month and year from the Sakila database.

    Parameters:
        engine (Engine): SQLAlchemy engine object.
        month (int): The month for which rental data is to be retrieved.
        year (int): The year for which rental data is to be retrieved.

    Returns:
        pd.DataFrame: DataFrame containing the rental data for the specified month and year.
    """
    # Use text() for compatibility
    query = text("""
    SELECT *
    FROM rental
    WHERE MONTH(rental_date) = :month AND YEAR(rental_date) = :year
    """)

    # Execute the query
    with engine.connect() as connection:
        rentals_df = pd.read_sql_query(sql=query, con=connection, params={"month": month, "year": year})

    return rentals_df


In [ ]:
from sqlalchemy.engine import Engine
from typing import Tuple
from sqlalchemy.sql import text

def get_date_range(engine: Engine) -> Tuple[str, str]:
    """
    Determines the range of dates in the rental table of the Sakila database.

    Parameters:
        engine (Engine): SQLAlchemy engine object.

    Returns:
        Tuple[str, str]: A tuple containing the earliest and latest rental dates (earliest_date, latest_date) as strings.
    """
    query = text("""
    SELECT MIN(rental_date) AS earliest_date, MAX(rental_date) AS latest_date
    FROM rental
    """)

    with engine.connect() as connection:
        result = connection.execute(query).fetchone()

    # access tuple index
    return result[0], result[1]

earliest_date, latest_date = get_date_range(engine)
print(f"The rental data ranges from {earliest_date} to {latest_date}.")


In [ ]:
# Retrieve rentals for July 2005
rentals_jul_2005 = rentals_month(engine, 7, 2005)

# Display the resulting DataFrame
print(rentals_jul_2005)

# Retrieve rentals for August 2005
rentals_aug_2005 = rentals_month(engine, 8, 2005)

# Display the resulting DataFrame
print(rentals_aug_2005)

Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".

Hint: Consider making use of pandas groupby()

In [27]:
# Note: I noticed a potential inconsistency in the questions regarding the rental_count_month function.
# If the rentals_month function already returns data for a specific month and year, why does rental_count_month
# need to take month and year as additional inputs? It seems redundant since the input DataFrame from rentals_month 
# would already be filtered for the specified time period.

# I believe rental_count_month could simply take the filtered DataFrame as input and dynamically infer 
# the column name based on the month and year within the data. 


import pandas as pd

def rental_count_month(rentals_df: pd.DataFrame, month: int, year: int) -> pd.DataFrame:
    """
    Counts the number of rentals made by each customer_id during the selected month and year.

    Parameters:
        rentals_df (pd.DataFrame): DataFrame containing rental data for a specific month and year.
        month (int): The month for which rental data is being analyzed (used for column naming).
        year (int): The year for which rental data is being analyzed (used for column naming).

    Returns:
        pd.DataFrame: DataFrame containing customer_id and their rental counts with a dynamically named column.
    """
    column_name = f"rentals_{str(month).zfill(2)}_{year}"

    # Group by customer_id and count the number of rentals
    rental_counts = rentals_df.groupby('customer_id').size().reset_index(name=column_name)

    return rental_counts


rental_counts_month_jul_2005_df = rental_count_month(rentals_jul_2005, 7, 2005)
print(rental_counts_month_jul_2005_df)


Object `parameters` not found.
     customer_id  rentals_07_2005
0              1               12
1              2               14
2              3               13
3              4                5
4              5               16
..           ...              ...
594          595               19
595          596                6
596          597                7
597          598               16
598          599                7

[599 rows x 2 columns]


In [28]:
import pandas as pd

def compare_rentals(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    """
    Compares the number of rentals made by each customer in two different months
    and returns a combined DataFrame with a 'difference' column.

    Parameters:
        df1 (pd.DataFrame): DataFrame containing rental counts for the first month/year.
        df2 (pd.DataFrame): DataFrame containing rental counts for the second month/year.

    Returns:
        pd.DataFrame: A combined DataFrame with columns from both inputs and a 'difference' column.
    """
    # merge the two DataFrames on 'customer_id', filling missing values with 0
    combined_df = pd.merge(df1, df2, on='customer_id', how='outer', suffixes=('_month1', '_month2')).fillna(0)
    
    # calculate the difference between the two months
    rental_columns = [col for col in combined_df.columns if col.startswith('rentals_')]
    combined_df['difference'] = combined_df[rental_columns[0]] - combined_df[rental_columns[1]]
    
    return combined_df


rentals_jul_2005 = rentals_month(engine, 7, 2005)
rentals_aug_2005 = rentals_month(engine, 8, 2005)

rental_counts_month_jul_2005_df = rental_count_month(rentals_jul_2005, 7, 2005)
rental_counts_month_aug_2005_df = rental_count_month(rentals_aug_2005, 8, 2005)



compared_rentals_df = compare_rentals(rental_counts_month_jul_2005_df, rental_counts_month_aug_2005_df)

print(compared_rentals_df)



     customer_id  rentals_07_2005  rentals_08_2005  difference
0              1               12               11           1
1              2               14               11           3
2              3               13                7           6
3              4                5               11          -6
4              5               16               13           3
..           ...              ...              ...         ...
594          595               19                8          11
595          596                6               13          -7
596          597                7               12          -5
597          598               16                5          11
598          599                7                7           0

[599 rows x 4 columns]
